# $\tau$ VS $\eta$ (standard deviation of the Gaussian kernel versus normalized cut-off frequency)

In [ ]:
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
from denoising.utils import get_gaussian_kernel

# Define parameters
sigma_values = np.linspace(0.5, 10, 100)  # Range of sigma values to test
N = 256  # Signal length
normalized_cutoff_freqs = []

for sigma in sigma_values:
    # Generate Gaussian kernel
    kernel = get_gaussian_kernel(sigma)

    # Compute its frequency response
    freq_response = np.abs(np.fft.fft(kernel, N))  # Compute FFT
    freq_response = freq_response[:N//2]  # Take only the first half (positive frequencies)
    freq_axis = np.linspace(0, 0.5, N//2)  # Normalized frequency axis (Nyquist = 0.5)

    # Find the effective cut-off frequency (where magnitude falls below ~0.707)
    cutoff_idx = np.where(freq_response < freq_response[0] / np.sqrt(2))[0][0]
    cutoff_freq = freq_axis[cutoff_idx]
    normalized_cutoff_freqs.append(cutoff_freq)

# Fit an empirical formula: f_c^* ≈ C / sigma
coeff = np.polyfit(1 / sigma_values, normalized_cutoff_freqs, 1)

# Display empirical relationship
plt.figure(figsize=(6, 4))
plt.plot(sigma_values, normalized_cutoff_freqs, 'bo-', label='Measured Cut-off')
plt.plot(sigma_values, coeff[0] * (1 / sigma_values) + coeff[1], 'r--', label=f'Fit: {coeff[0]:.3f}/sigma + {coeff[1]:.3f}')
plt.xlabel("Sigma")
plt.ylabel(r"Effective Normalized Cut-off Frequency ($f_c^*$)")
plt.legend()
plt.title("Empirical Relationship Between Sigma and Cut-off Frequency")
plt.grid()
plt.show()

print(f"Empirical formula: f_c^* ≈ {coeff[0]:.3f} / sigma + {coeff[1]:.3f}")


In [ ]:
def normalized_cutoff_to_tau(cutoff_freq):
    tau = 0.141 / cutoff_freq
    return tau

In [ ]:
tau = normalized_cutoff_to_tau(0.25)
tau